# 실습 3. RNN을 이용한 😀감정분석😑 모델 학습하기



<b>학습 목표:    
- NLU 모델링을 위한 데이터 전처리(토크나이징 & 인코딩, 라벨 만들기)를 이해하고 Python을 사용해 코딩한다.
</b>








## #0. 실습 준비하기
먼저 구글 드라이브를 마운트하고, 필요한 라이브러리를 설치하겠습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
""" 한국어 형태소 분석 라이브러리 """
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
     |████████████████████████████████| 3.8MB 41.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

'2.3.0'

깃허브에 있는 감성분석 데이터셋을 다운로드해 읽어오겠습니다.

In [ ]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2020-10-06 07:39:48--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  30.2MB/s    in 0.5s    

2020-10-06 07:39:49 (30.2 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2020-10-06 07:39:49--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘rat

In [ ]:
""" 데이터 읽어오기 """
with open("ratings_train.txt") as f:
  raw_train = f.readlines()
with open("ratings_test.txt") as f:
  raw_test = f.readlines()
raw_train = [t.split('\t') for t in raw_train[1:]]
raw_test = [t.split('\t') for t in raw_test[1:]]

FULL_TRAIN = []
for line in raw_train:
  if int(line[2].strip()) == 0:
    FULL_TRAIN.append([line[0], line[1], "부정"])
  elif int(line[2].strip()) == 1:
    FULL_TRAIN.append([line[0], line[1], "긍정"])
FULL_TEST = []
for line in raw_test:
  if int(line[2].strip()) == 0:
    FULL_TEST.append([line[0], line[1], "부정"])
  elif int(line[2].strip()) == 1:
    FULL_TEST.append([line[0], line[1], "긍정"])

def print_stat(name, data):
  print("{:<10}: {}개 (긍정 {}, 부정 {})".format(name, len(data), len([t for t in data if t[2]=="긍정"]), len(data)-len([t for t in data if t[2]=="긍정"])))

print_stat("FULL_TRAIN", FULL_TRAIN)
print_stat("FULL_TEST", FULL_TEST)

FULL_TRAIN: 150000개 (긍정 74827, 부정 75173)
FULL_TEST : 50000개 (긍정 25173, 부정 24827)


In [ ]:
# 데이터 예시 : id, 문장, 라벨 순서
FULL_TRAIN[:10]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '부정'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '긍정'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '부정'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '부정'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '긍정'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '부정'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '부정'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '부정'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '긍정'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '긍정']]

이제 데이터를 train / validation / test 셋으로 나누겠습니다.   
학습 시간관계상 train 50000건, validation 10000건, test 10000건만 샘플링해서 사용하겠습니다.

In [ ]:
import random

random.seed(1)
random.shuffle(FULL_TRAIN)
random.shuffle(FULL_TEST)
train = FULL_TRAIN[:50000]
val = FULL_TRAIN[50000:60000]
test = FULL_TEST[:10000]

print_stat("train", train)
print(".. ex:", train[0])
print_stat("validation", val)
print(".. ex:", val[0])
print_stat("test", test)
print(".. ex:", test[0])

train     : 50000개 (긍정 24914, 부정 25086)
.. ex: ['7570203', '장쯔이 그때나 지금이나 이뻤다', '긍정']
validation: 10000개 (긍정 4962, 부정 5038)
.. ex: ['8413570', '거지 발싸개같은 영화도 자막 파일이 넘쳐나는게 허다한데, 왜 이 좋은 영화는 자막을 구할수가 없을까.. 60, 70년대의 프랑스, 이탈리아, 독일의 수작들이 자막이 없어서 수OO이 지난 지금도 일반 대중에게 제대로 알려지지 못했다.', '긍정']
test      : 10000개 (긍정 5027, 부정 4973)
.. ex: ['1458790', '허우 샤오시엔 작품은 모두 만점!', '긍정']


In [ ]:
## 나누어 놓은 train/ validation/ test 데이터 저장하기
import json

with open("/content/gdrive/My Drive/NLP/Sentiment_train.json", "w") as f:
  f.write(json.dumps(train))
with open("/content/gdrive/My Drive/NLP/Sentiment_val.json", "w") as f:
  f.write(json.dumps(val))
with open("/content/gdrive/My Drive/NLP/Sentiment_test.json", "w") as f:
  f.write(json.dumps(test))

# #1. 토크나이징

<img src = "https://github.com/seungyounglim/temporary/blob/master/fig_step1.PNG?raw=true">

먼저 Komoran 형태소분석기를 사용해 형태소 분석 함수 tokenize를 정의하겠습니다.

In [ ]:
from konlpy.tag import Komoran

komoran = Komoran()

def tokenize(sentence):
  return komoran.morphs(sentence)

tokenize("미션 완료!!")

['미션', '완료', '!!']

# #2. 단어 사전 로딩 & 인코딩
이제 인코딩을 위해 단어사전을 로딩하겠습니다.   
실습 소개에서 말씀드렸던 것처럼 CBOW 학습에서 사용한 70002개의 토큰에    
이번 태스크 수행에서 새롭게 나온 토큰을 추가하겠습니다.
<img src = "https://github.com/seungyounglim/temporary/blob/master/fig_step2.PNG?raw=true">   


#### Step 1. 단어사전 로딩하기
- 기 학습된 단어 임베딩을 불러와 사용
- 도메인 특화적인 단어로 [UNK]가 되는 토큰들을 식별해 단어 사전에 추가

In [ ]:
""" 지난 실슬 때 만들었던 단어 사전 로딩 """

## final_embeddings: 70002개 토큰에 대한 워드 벡터 매트릭스 shape=(70002, 128)
## vocab_list: 위의 워드 벡터와 매칭되는 단어 사전

with open("/content/gdrive/My Drive/NLP/vecs.tsv") as f:
  vecs = [v.strip() for v in f.readlines()]
final_embeddings = [v.split("\t") for v in vecs]
final_embeddings = np.array(final_embeddings, dtype="float32")

with open("/content/gdrive/My Drive/NLP/meta.tsv") as f:
  vocab_list = [v.strip() for v in f.readlines()]

print("** 단어 사전 개수:", len(vocab_list))
print("** 단어벡터 예시: ")
print("token :", vocab_list[999])
print("vector:", final_embeddings[999])

** 단어 사전 개수: 70002
** 단어벡터 예시: 
token : 약속
vector: [ 1.39787421e-01 -3.84234697e-01 -5.40808678e-01  1.47639737e-02
 -1.44524083e-01  7.03318939e-02  2.15433896e-01  3.47744254e-03
 -1.11955218e-02  4.17765081e-01  2.20791101e-01  4.46164668e-01
 -5.74833632e-01 -4.71866518e-01 -4.32424575e-01  2.22987652e-01
  1.94115311e-01  2.63385952e-01  3.28183919e-01 -4.22089607e-01
 -1.04542814e-01  1.62697881e-01 -1.58821136e-01 -3.23788792e-01
 -5.71838796e-01  5.85799575e-01 -7.10058093e-01  1.43308431e-01
 -4.11221199e-02  2.94368807e-02  2.57899791e-01 -4.58910733e-01
  1.42146766e-01 -3.38556468e-01  1.25622004e-01 -1.85091704e-01
  4.02939171e-01 -1.01323523e-01  2.85541564e-01  2.12628245e-01
 -4.62328009e-02  1.20681420e-01 -1.23937130e-01  1.05506694e-02
  3.15133363e-01 -2.08656356e-01  7.68174082e-02 -1.84903651e-01
  1.83630511e-01 -2.24744290e-01  4.72059786e-01  4.76261348e-01
  2.46252075e-01  2.08968930e-02  1.21434242e-01  3.05238903e-01
  9.10391510e-01 -5.51669121e-01  3.258

<font color="blue"> 🙋‍♀️[QUIZ] final_embeddings의 차원이 70002 x 128인 이유는 무엇인가요?

현재 태스크에서 [UNK]로 떨어지는 단어가 어떤 것들이 있는지 살펴보겠습니다.   
CBOW 실습에서 사용했던 collections의 Counter()기능을 사용하겠습니다.

In [ ]:
import collections 
from tqdm import tqdm
tot_tokens = 0
oov_counter = collections.Counter() #새로운 토큰 카운터

Tokenized_train = [] #토크나이징된 데이터 저장
for dat in tqdm(train):
  sent = dat[1]
  tokenized_sent = tokenize(sent)
  tot_tokens += len(tokenized_sent) #토큰 개수
  for word in tokenized_sent:
    if word not in vocab_list: #기존 단어사전에서 찾을 수 없으면
      oov_counter[word] += 1 #Counter에서 개수 세기
      
  Tokenized_train.append([dat[0],tokenized_sent, dat[2]]) # 토크나이징된 문장 저장

100%|██████████| 50000/50000 [02:00<00:00, 415.74it/s]


In [ ]:
print("# OOV Tokens:", len(oov_counter))
print("{}/{} ({:.2f}%) are [UNK] in train tokens".format(sum(oov_counter.values()) , tot_tokens , 100*sum(oov_counter.values())/tot_tokens))

# OOV Tokens: 15927
36292/955855 (3.80%) are [UNK] in train tokens


👉 50,000개의 train 문장에서 무려 15,000개 이상의 Out-Of-Vocabulary 토큰들이 발견되었습니다.    
🙋‍♀️ 어떤 토큰들이 기존 단어 사전에 포함되어 있지 않았을까요?

In [ ]:
most_common = oov_counter.most_common(len(oov_counter))
print(most_common[:10])
print(most_common[-10:])

[('ㅋㅋ', 647), ('......', 601), ('♥', 472), ('ㅋㅋㅋ', 358), ('ㅋ', 358), ('ㅠㅠ', 321), ('이쁘', 314), ('ㅎㅎ', 297), ('막장', 260), ('ㅡㅡ', 234)]
[('스토리임....ㅡㅡ', 1), ('어렷을떄', 1), ('섬세하뮤ㅠ', 1), ('없슴다', 1), ('hhhh', 1), ('린날', 1), ('너거', 1), ('재밋었을듯..', 1), ('장음', 1), ('죽다ㅠ_ㅠ', 1)]


예상했던 것과 같이 구어체적인 표현들이 새로 등장한 것을 볼 수 있습니다.   
이제 새로 나온 토큰들을 기존의 단어사전에 추가해 감성분석 태스크 분석을 위한 새로운 단어사전을 만들겠습니다.

In [ ]:
### Train 데이터에서 새로 발견한 토큰을 기존 단어 사전에 추가
new_vocab_list = vocab_list.copy() # 1. 기존 단어 사전 복사
new_vocab_list.extend([v[0] for v in most_common]) # 2. 새로 나온 토큰을 기존 리스트에 이어붙이기
print("# New Vocabs = {}".format(len(new_vocab_list)))

# New Vocabs = 85929


<font color="blue"> 🙋‍♀️[QUIZ] 새로 나온 토큰을 기존 리스트의 뒤에 이어붙여야 하는 이유는 무엇인가요?   
🙋‍♀️[QUIZ] 새로운 단어사전은 몇 개의 토큰이 있나요? 모델에서 임베딩 차원은 몇 차원이 되어야 하나요?

새로 만든 단어사전을 저장해 놓아야 이후 학습~추론에서 사용할 수 있겠지요? 

In [ ]:
## 새로 만든 단어사전 저장
with open("/content/gdrive/My Drive/NLP/Sentiment_vocab.json", "w") as f:
  f.write(json.dumps(new_vocab_list))

#### Step 2. 데이터 토크나이징하기

그럼 validation과 test 데이터에 대해서도 토크나이징을 진행하겠습니다. 

In [ ]:
Tokenized_val = [] #토크나이징된 데이터 저장
for dat in tqdm(val):
  tokenized_sent = tokenize(dat[1])   
  Tokenized_val.append([dat[0],tokenized_sent, dat[2]]) # 토크나이징된 문장 저장

Tokenized_test = [] #토크나이징된 데이터 저장
for dat in tqdm(test):
  tokenized_sent = tokenize(dat[1])   
  Tokenized_test.append([dat[0],tokenized_sent, dat[2]]) # 토크나이징된 문장 저장

100%|██████████| 10000/10000 [00:07<00:00, 1288.90it/s]


In [ ]:
print(len(Tokenized_train) , len(Tokenized_val), len(Tokenized_test))

50000 10000 10000


#### Step 3. 텍스트 인코더 코딩하기

토크나이징된 문장을 인덱스로 바꾸기 위해, 지난 모듈에서 정의했던 TextEncoder를 사용하겠습니다.    
지난 시간에 코딩했던 내용을 utils.py에 담아두었습니다.   
여기서 TextEncoder를 불러와 사용하도록 하겠습니다.

In [ ]:
from utils import TextEncoder
text_encoder = TextEncoder(new_vocab_list)

In [ ]:
## 토크나이저 테스트
sent = "새로 만든 단어사전 ♥"
tokenized_sent = tokenize(sent)
tokenized_ids = text_encoder.convert_tokens_to_ids(tokenized_sent) # 토큰 -> 인덱스
reversed_token= text_encoder.convert_ids_to_tokens(tokenized_ids) # 인덱스 -> 토큰

print(tokenized_ids)
print(reversed_token)

[1339, 128, 9, 1313, 1815, 70004]
['새로', '만들', 'ㄴ', '단어', '사전', '♥']


### Step 4. 태스크 수행을 위한 형태로 만들기   
그럼 이제 본격적으로 감성분석 태스크를 수행할 수 있는 형태로 인풋을 준비하겠습니다.   

이를 위해 아래의 세 가지를 수행해야 합니다.   
1. 인풋 자연어 토큰을 정수 인덱스로 변환(@text_encoder.convert_tokens_to_ids)
2. 정답 라벨을 정수 인덱스로 변환 -> 라벨 매핑 사전 필요   
3. 배치 처리를 위해 패딩 & numpy array로 변환


위의 세 가지 작업을 수행하는 함수로 create_cls_feature이라는 함수를 정의하겠습니다.    
TextEncoder와 마찬가지로, 이 함수는 다양한 텍스트 분류 과제에서 코드 재활용이 가능합니다. 

create_cls_feature
- 함수 인풋: 데이터 , text_encoder, max_seq_len
- 함수 아웃풋: input_ids , labels, label_map   



그럼 데이터 example 일부에 대해 한 단계씩 차례대로 수행하며 차례대로 코딩해보겠습니다.

<b>1. 인풋 자연어 토큰을 정수 인덱스로 변환(@text_encoder.convert_tokens_to_ids)   </b>   
함수는 인풋으로 Tokenized_train과 같은 데이터를 받습니다.   
데이터는 리스트의 리스트로 이루어져 있으며, 각각의 리스트는 [문장 번호, 토큰화된 문장, 정답 라벨]의 모양으로 되어 있습니다.   
(예) ['7570203', ['장쯔이', '그때', '나', '지금', '이나', '이뻤다'], '긍정']

In [ ]:
examples = Tokenized_train[:3]

for example in examples:
  print(example)

['7570203', ['장쯔이', '그때', '나', '지금', '이나', '이뻤다'], '긍정']
['6317334', ['음식', '가지', '고', '사람', '을', '해치', 'ㄴ', '사람', '이', '명장', '되', '고', ',', '인', '주', '는', '끝', '까지', '반성', '도', '없', '구', ',', '막장', '드라마', '로', '기억', '되', 'ㄹ', '듯'], '부정']
['6287811', ['뭔내용인지도모르겟고', '돈', '아깝', '고'], '부정']


<font color="red"> [MISSION] examples에 있는 토큰화된 자연어 문장들을 인덱스로 변환해 input_ids라는 리스트에 저장해보세요.

In [ ]:
input_ids = []
 
for example in examples:
  idx, tokenized_sent, label = example
  input_id = text_encoder.convert_tokens_to_ids(tokenized_sent)  ## [★ CODE ★]
  input_ids.append(input_id)
 
for i, input_id in enumerate(input_ids):
  print("문장 {}:".format(i), input_id)

문장 0: [34556, 1862, 75, 467, 150, 71299]
문장 1: [2103, 110, 16, 111, 7, 12533, 9, 111, 3, 11094, 17, 16, 11, 435, 62, 6, 540, 61, 5730, 31, 58, 562, 11, 70010, 526, 24, 1478, 17, 32, 1036]
문장 2: [73219, 762, 11267, 16]


<b>2. 정답 라벨을 정수 인덱스로 변환 -> 라벨 매핑 사전 필요    </b>   

다음으로 모델에게 라벨을 알려줄 수 있는 라벨 매핑 사전이 필요합니다.    
모델은 "긍정"이나 "부정" 이라는 단어를 인식할 수 있기 때문에, 각 라벨에 해당하는 정수 인덱스를 매핑해주는 것입니다.   
{"긍정": 0 , "부정": 1} 이런 식으로요.   

이 때 라벨 매핑 사전은 학습~추론에서 유지되어야 하겠지요?   
따라서 학습 데이터를 이용해   
1. 새로 발견된 라벨을 라벨 매핑 사전에 저장한다.
2. 완성된 라벨 매핑 사전을 저장해둔다.   

이후 검증/ 테스트 데이터에서는 위에서 만든 라벨 매핑 사전을 사용해 매핑만 해주도록 코드를 구현하겠습니다. 

In [ ]:
CREATE_LABEL_MAP = True # 새로 사전을 만들겠다. 
label_map = {}
label_index = 0 

label_ids = [] ## 데이터에 대해 변환된 라벨 저장
for example in examples:
  idx, tokenized_sent, label = example
  if label not in label_map: # 라벨이 label_map에 없다면
    if CREATE_LABEL_MAP:
      label_map[label] = label_index # 라벨 맵에 해당 라벨 추가
      label_index += 1
      label_id = label_map[label]      
    else:
      ## train 이외의 단계에서 새로운 라벨이 발견되었다면, 이는 잘못된 데이터입니다. 
      ## 따라서 Error 메시지를 추가하고, 건너뛰도록 하겠습니다.
      print("** ERROR: UNSEEN LABEL DETECTED -", label)
      continue
  else: # 라벨을 찾았으면
    label_id = label_map[label]
  label_ids.append(label_id)

print("라벨 매핑 사전:", label_map)
print("-> 원래 라벨:", [e[2] for e in examples])
print("-> 인덱싱 후:", label_ids)

라벨 매핑 사전: {'긍정': 0, '부정': 1}
-> 원래 라벨: ['긍정', '부정', '부정']
-> 인덱싱 후: [0, 1, 1]


<b>3. 배치 처리를 위해 패딩 & numpy array로 변환    </b>   

정수 인덱스로 변환한 input_ids를 살펴볼까요?

In [ ]:
for i, input_id in enumerate(input_ids):
  print("문장 {}:".format(i), input_id)

문장 0: [34556, 1862, 75, 467, 150, 71299]
문장 1: [2103, 110, 16, 111, 7, 12533, 9, 111, 3, 11094, 17, 16, 11, 435, 62, 6, 540, 61, 5730, 31, 58, 562, 11, 70010, 526, 24, 1478, 17, 32, 1036]
문장 2: [73219, 762, 11267, 16]


👉자연어 문장은 길이가 제각각이기 때문에 각 문장의 길이가 제각각인 것을 볼 수 있습니다.   

하지만 딥러닝 배치 처리를 위해서는 이들을 일정한 길이로 맞춰주어야 하지요.   
이 기능을 수행해 주는 것이 tensorflow.keras.preprocessing.sequence의 pad_sequences 기능입니다. 

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_seq_len = 20

input_ids_1 = pad_sequences(input_ids, 
                            maxlen=max_seq_len, # 최대 문장 길이
                            padding="post", # 패딩은 문장의 뒤에 한다
                            truncating="post" ) # 최대 길이를 넘어갈 경우 뒷 부분을 자른다
print("** 예시 1:")
print(input_ids_1)
input_ids_2 = pad_sequences(input_ids, 
                            maxlen=max_seq_len, # 최대 문장 길이
                            padding="post", # 패딩은 문장의 앞에 한다
                            truncating="pre" ) # # 최대 길이를 넘어갈 경우 앞 부분을 자른다
print("\n** 예시 2:")
print(input_ids_2)

** 예시 1:
[[34556  1862    75   467   150 71299     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [ 2103   110    16   111     7 12533     9   111     3 11094    17    16
     11   435    62     6   540    61  5730    31]
 [73219   762 11267    16     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]

** 예시 2:
[[34556  1862    75   467   150 71299     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [   17    16    11   435    62     6   540    61  5730    31    58   562
     11 70010   526    24  1478    17    32  1036]
 [73219   762 11267    16     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]


<font color="red"> [MISSION] 한국어에서 핵심적인 정보는 문장의 뒷 부분에 있다고 판단하였습니다.   
따라서, 문장이 max_seq_len을 넘어가면 앞부분을 자르고, 모자르면 문장 뒤에 0 패딩을 하기로 결정하였습니다.   
pad_sequences 함수를 사용해 이 기능을 수행할 수 있도록 직접 코딩해보세요

In [ ]:
input_ids = pad_sequences(input_ids,
                            maxlen=max_seq_len,
                            padding="post",
                            truncating="pre")
 
print(input_ids)

그럼 위에서 하나씩 만들었던 함수를 모두 합쳐서 create_cls_feature 함수를 만들겠습니다.   
이 함수는 utils.py에도 저장되어 있어, 향후에도 텍스트 분석에서 활용할 수 있습니다.

In [ ]:
from tqdm import tqdm
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
def create_cls_feature(examples, text_encoder, max_seq_len, label_map=None):

  input_ids = [] # 정수 인덱스로 변환한 문장들의 리스트
  labels = [] # 정답 라벨 리스트

  if label_map is None: #label_map이 없으면 -> 이번에 데이터를 처리하면서 새로 생성함
    CREATE_LABEL_MAP = True
    label_map = {}
    label_index = 0
  else: 
    CREATE_LABEL_MAP = False
    print("** Start creating features using label map")
    print(label_map)

  for example in examples:
    idx, tokenized_sent, label = example

    ## 1. text_encoder 사용해 정수로 변환 
    input_id = text_encoder.convert_tokens_to_ids(tokenized_sent)
    if len(input_id) == 0:
      print("Sentence with length = 0... continue", example)
      continue

    ## 2. label 매핑 & index로 변환
    if label in label_map:
      label_id = label_map[label]
    else:
      if CREATE_LABEL_MAP:
        # label map에 추가
        label_map[label] = label_index
        label_index += 1
        label_id = label_map[label]
      else:
        print("** ERROR: UNSEEN LABEL DETECTED -", label)
        continue
  
    ## 전체 리스트에 append
    input_ids.append(input_id)
    labels.append(label_id)
    
      
  """ max_seq_len을 넘는 문장은 절단, 모자르는 것은 PADDING """
  input_ids = pad_sequences(input_ids, 
                            maxlen=max_seq_len, 
                            padding="post", 
                            truncating="pre")

  ## np.array로 변환해 리턴
  input_ids = np.array(input_ids)
  labels = np.array(labels)
  
  assert len(input_ids) == len(labels)
  print("** {} examples processed".format(len(input_ids)))

  return input_ids, labels, label_map


함수를 사용해 train, validation, test 데이터를 각각 처리하겠습니다.   
train 데이터를 만들 때는 label_map을 인풋에 주지 않아, 전처리중 label_map이 생성되도록 합니다.   
validation과 test 데이터를 만들 때에는 학습 데이터에 대해 만들어진 label_map을 사용해 전처리를 진행합니다. 

In [ ]:
# TRAIN
train_ids, train_labels, label_map = create_cls_feature(Tokenized_train, text_encoder, max_seq_len=50, label_map = None)
# VAL
val_ids, val_labels, _ = create_cls_feature(Tokenized_val, text_encoder, max_seq_len=50, label_map = label_map)
# TEST
test_ids, test_labels, _ = create_cls_feature(Tokenized_test, text_encoder, max_seq_len=50, label_map = label_map)

Sentence with length = 0... continue ['5942978', [], '부정']
** 49999 examples processed
** Start creating features using label map
{'긍정': 0, '부정': 1}
Sentence with length = 0... continue ['2172111', [], '긍정']
** 9999 examples processed
** Start creating features using label map
{'긍정': 0, '부정': 1}
** 10000 examples processed


In [ ]:
print("# Train={} # Val={} # Test={}".format(len(train_ids), len(val_ids), len(test_ids)))

# Train=49999 # Val=9999 # Test=10000


모델링을 위한 데이터 전처리가 완료되었습니다!    
그럼 데이터를 저장하고, 다음 페이지에서 모델링을 계속 진행하겠습니다.

In [ ]:
import pickle
prepro_data = {
    "train_ids": train_ids,
    "train_labels": train_labels,
    "val_ids": val_ids,
    "val_labels": val_labels,
    "test_ids": test_ids,
    "test_labels": test_labels,
    "label_map":label_map
}

with open("/content/gdrive/My Drive/NLP/Sentiment_prepro_data.pkl", "wb") as f:
  pickle.dump(prepro_data, f)

utils.py 파일은 드라이브에 저장해, 향후 활용할 수 있도록 하겠습니다. 

In [ ]:
!cp utils.py "/content/gdrive/My Drive/NLP/"